code citation:

google colab unet regression
https://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb#scrollTo=FyRpvwENxE-A

Ryan colab demo
https://colab.research.google.com/drive/1opTXxC2NvlVZ3EW1l45dz9wNMztiE2I_?usp=sharing#scrollTo=QaGLx6PtzQhf



# This notebook contains variables that may be used by multiple other notebooks

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import geemap
import folium

2023-08-16 00:33:37.100151: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: U

## Define feature list and bands

In [3]:
feature_list = []
# Fire label
# 2000-2023
# https://developers.google.com/earth-engine/datasets/catalog/FIRMS
fire_map_info = {'name':'FIRMS', 'band' :'T21', 'min': 300, 'max': 509.29}
# MODIS Normalized difference vegetation index NDVI dataset
# 2000-2023
# https://developers.google.com/earth-engine/datasets/catalog/MODIS_MOD09GA_006_NDVI
veg_map_info = {'name':'MODIS/MOD09GA_006_NDVI', 'band' :'NDVI', 'min': -1, 'max': 1}
feature_list.append(veg_map_info)
# MODIS Leaf Area Index/FPAR 4-Day Global 500m
# 2002-2023
# https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MCD15A3H
# upper limit is 20+, so set max = 30
leaf_map_info = {'name':'MODIS/061/MCD15A3H', 'band' :'Lai', 'min': 0, 'max': 30}
feature_list.append(leaf_map_info)
# FLDAS: Famine Early Warning Systems Network (FEWS NET) Land Data Assimilation System
# 1982-2023
# https://developers.google.com/earth-engine/datasets/catalog/NASA_FLDAS_NOAH01_C_GL_M_V001
soil_moist_map_info = {'name':'NASA/FLDAS/NOAH01/C/GL/M/V001', 'band' :'SoilMoi00_10cm_tavg', 'min': 0, 'max': 1}
feature_list.append(soil_moist_map_info)
# Terra Land Surface Temperature and Emissivity Daily Global 1km
# 2000-2023
# https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD11A1
temp_map_info = {'name':'MODIS/061/MOD11A1', 'band' :'LST_Day_1km', 'min': 13658, 'max': 15658}
feature_list.append(temp_map_info)
# ERA5 Monthly Aggregates dataset
# 1979-2020
# https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_MONTHLY
precipitation_map_info = {'name':'ECMWF/ERA5/MONTHLY', 'band' :'total_precipitation', 'min': 0, 'max': 0.4}
feature_list.append(precipitation_map_info)
wind_speed_u_map_info = {'name':'ECMWF/ERA5/MONTHLY', 'band' :'u_component_of_wind_10m', 'min': -8.7, 'max': 8.7}
feature_list.append(wind_speed_u_map_info)
wind_speed_v_map_info = {'name':'ECMWF/ERA5/MONTHLY', 'band' :'v_component_of_wind_10m', 'min': -6.8, 'max': 6.8}
feature_list.append(wind_speed_v_map_info)

# Copernicus Global Land Cover Layers: CGLS-LC100 Collection 3
# 2015-end2019
# https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_Landcover_100m_Proba-V-C3_Global
land_cover_info = {'name':"COPERNICUS/Landcover/100m/Proba-V-C3/Global", 'band': 'discrete_classification', 'min': 0, 'max': 200}
# feature_list.append(wind_speed_v_map_info)
# print(feature_list)
print(len(feature_list))
print(type(feature_list))

7
<class 'list'>


In [4]:
LABEL = fire_map_info['band']
BANDS = []
for feature in feature_list:
  BANDS += [feature['band']]
FEATURE_NAMES = BANDS + [LABEL]
print(FEATURE_NAMES)

['NDVI', 'Lai', 'SoilMoi00_10cm_tavg', 'LST_Day_1km', 'total_precipitation', 'u_component_of_wind_10m', 'v_component_of_wind_10m', 'T21']


## Define Google Cloud Storage relevant data

In [5]:
PROJECT = 'ee-my-char'
DATA_BUCKET = '6140-data-bucket'
OUTPUT_BUCKET = '6140-output-bucket'
# Specify names locations for outputs in Cloud Storage.
FOLDER = 'fcnn-demo'
TRAINING_BASE = 'training_patches'
EVAL_BASE = 'eval_patches'

## Define kernel and training relevant data

In [6]:
# Specify the size and shape of patches expected by the model.
KERNEL_SIZE = 256
# KERNEL_SIZE = 128
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]

# List of fixed-length features, all of which are float32.
columns = [tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURE_NAMES]
FEATURES_DICT = dict(zip(FEATURE_NAMES, columns))

# Specify model training parameters.
BATCH_SIZE = 64
EPOCHS = 10
BUFFER_SIZE = 2000
OPTIMIZER = 'adam'
LOSS = 'MeanSquaredError'
# METRICS = ['RootMeanSquaredError']
# METRICS=[tf.keras.metrics.Recall(), tf.keras.metrics.Accuracy()]
METRICS = {
    'recall': tf.keras.metrics.Recall(),
    'accuracy': tf.keras.metrics.Accuracy()
}

2023-08-16 00:33:39.572688: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-16 00:33:39.582410: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-16 00:33:39.584295: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

cloud masking function

In [7]:
# https://colab.research.google.com/drive/1opTXxC2NvlVZ3EW1l45dz9wNMztiE2I_?usp=sharing#scrollTo=QaGLx6PtzQhf
def maskL8sr(image):
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()
  qa = image.select('pixel_qa')
  mask1 = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
    qa.bitwiseAnd(cloudsBitMask).eq(0))
  mask2 = image.mask().reduce('min')
  mask3 = image.select(opticalBands).gt(0).And(
          image.select(opticalBands).lt(10000)).reduce('min')
  mask = mask1.And(mask2).And(mask3)
  return image.select(opticalBands).divide(10000).addBands(
          image.select(thermalBands).divide(10).clamp(273.15, 373.15)
            .subtract(273.15).divide(100)).updateMask(mask)

If 'is_label', advance the time range by 'fire_pred_range', e.g. we use first 0-15 as feature and 16-30 as label

In [10]:
# Helper function avoid repeating
def getImageCollection(dataset, bounds, band):
  return ee.ImageCollection(dataset).filterBounds(bounds).select(band)

def getTrainImageCollection(map_info, is_label=False):
  img_collection = getImageCollection(map_info['name'],train_region_geometry, map_info['band']) # Get the image collection based on the provided map_info.
  img_list = ee.List([])

  advance_range = 0 if not is_label else fire_pred_range
  for yr in range(train_start_year, train_end_year+1):
    for mo in range(train_start_month, train_end_month + 1):
      # first half of a month
      start = ee.Date(f'{yr}-{mo:02d}-01').advance(advance_range, 'week')
      end = start.advance(2, 'week')
      date_filtered = img_collection.filterDate(start, end).map(maskL8sr) # Filter the image collection based on the date range.
      if not is_label:
        med = date_filtered.median().unitScale(map_info['min'], map_info['max']) # Calculate median and rescale.
      if is_label:
        med = date_filtered.max().unmask(0).divide(325).round().byte() # credit: david
      # print(med.getInfo())
      img_list = img_list.add(med)

      # second half of a month
      start = end
      end = start.advance(2, 'week')
      date_filtered = img_collection.filterDate(start, end).map(maskL8sr) # Filter the image collection based on the date range.
      if date_filtered.size().getInfo() > 0:  # Check if there are images in the second half of the month.
        # print("value exists from 15")
        if not is_label:
          med = date_filtered.median().unitScale(map_info['min'], map_info['max']) # Calculate median and rescale.
        if is_label:
          med = date_filtered.max().unmask(0).divide(325).round().byte() # credit: david
        # print(med.getInfo())
        
      img_list = img_list.add(med)

  return img_list